In [1]:
#Drive connection
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"

/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force&state=zL7RF7D46-x7xfRrJsS9U-X46Non7CmIyDpvqI3YcSM'
/bin/sh: 1: firefox: not found
/bin/sh: 1: google-chrome: not found
/bin/sh: 1: chromium-browser: not found
/bin/sh: 1: open: not found
Cannot retrieve auth tokens.
Failure("Error opening URL:https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.

In [1]:
!pip install git+https://github.com/NVIDIA/NeMo.git#egg=nemo_toolkit[all]
!pip install transformers
!pip install SpeechRecognition
!pip install wavio
!pip install jiwer

DEPRECATION: git+https://github.com/NVIDIA/NeMo.git#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/NeMo.git to /tmp/pip-install-3ubynq51/nemo-toolkit_825b040e31e246799d570fe4336748c5
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-3ubynq51/nemo-toolkit_825b040e31e246799d570fe4336748c5
  Resolved https://github.com/NVIDIA/NeMo.git to commit 6f2035bf985053bdda269f4df3b9338f5b58bfd5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.

In [49]:
!pip install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 9.3 MB/s eta 0:00:00


In [50]:
import pandas as pd
import pickle

import torch
import librosa
import wavio as wv
import nemo.collections.asr as nemo_asr
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import speech_recognition as sr
import jiwer
import warnings
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [52]:
#PreProcessed meta model
press_META_MODEL = "/content/gdrive/My Drive/Meta_Asr/Models/PreProcess_MetaModel.pkl"
#Raw meta model
RAW_META_Model = "/content/gdrive/My Drive/Meta_Asr/Models/Raw_MetaModel.pkl"
#define the candidate pre_trained models
ASR_PRE_TRAINED_MODELS = {
    0:"QuartZnet_Model",
    1:"Wav2Vec50_Model",
    2:"Wav2Vec960_Model"}

Wav2Vec50_Model = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
Wav2Vec960_Model = "facebook/wav2vec2-base-960h"
QuartZnet_Model = "QuartzNet15x5Base-En"
Speech2Text = sr.Recognizer()

#load Ground Truth  data
def load_data(path):
  df = pd.read_csv(path)
  return df


# Load the saved model
def load_model(model_name):
    with open(f"{model_name}", "rb") as file:
        model = pickle.load(file)
    return model

def predict_ASR_Model(df,best_model):
  X_test = df.drop(['Audio','selection_model_y','Target','GroundTruth_Text'],axis=1)
  # Compute predictions on the test set
  y_pred = best_model.predict(X_test)
  return y_pred

#resampling each audio to sample rate 16000
def resampling_audio(input_audio, target_sr=16000):
  temp_new_path="/content/rs_audio.wav"
  audio,sr = librosa.load(input_audio,sr=target_sr)
  wv.write(temp_new_path,audio,target_sr,sampwidth=2)
  return temp_new_path


#Wav2vec50 prediction
def wav2VecPrediction(resampled_audio):
    #load model
    model = Wav2Vec2ForCTC.from_pretrained(Wav2Vec50_Model )
    tokenizer = Wav2Vec2Tokenizer.from_pretrained(Wav2Vec50_Model)
    #preprocessing signzl
    audio, rate = librosa.load(resampled_audio)
    # to tokens
    input_values = tokenizer(audio, return_tensors = "pt").input_values
    logits = model(input_values).logits
    prediction = torch.argmax(logits, dim = -1)
    # decode results
    transcription = tokenizer.batch_decode(prediction)[0]
    #wer_Wav2Vec = jiwer.wer(transcription,real_text)
    return transcription

#prediction with wav2vec_690
def wav2Vec_960_Prediction(resampled_audio):
  #load model
  model = Wav2Vec2ForCTC.from_pretrained(Wav2Vec960_Model)
  tokenizer = Wav2Vec2Tokenizer.from_pretrained(Wav2Vec960_Model)
  #preprocessing signzl
  audio, rate = librosa.load(resampled_audio)
  # to tokens
  input_values = tokenizer(audio, return_tensors = "pt").input_values
  logits = model(input_values).logits
  prediction = torch.argmax(logits, dim = -1)
  # decode results
  transcription = tokenizer.batch_decode(prediction)[0]
  return transcription

#QuartZnet prediction
def quartZnetPrediction(resampled_audio):
  # Load Nemo ASR pre-trained model
  model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name=QuartZnet_Model)
  files = [resampled_audio]
  for fname, transcription in zip(files, model.transcribe(paths2audio_files=files)):
    raw_text = transcription
  return raw_text

#speech2Text prediction
def speech2textPrediction(resampled_audio):
  audio, sr_lib = librosa.load(resampled_audio)
  with sr.AudioFile(resampled_audio) as source:
    audio =  Speech2Text.record(source)
    text =  Speech2Text.recognize_google(audio)
  return text

#model evaluation using WER
def evaluate_model(prediction_text,real_text):
  wer_res = jiwer.wer(prediction_text,real_text)
  return wer_res


#Meta_asr prediction
def meta_Model_transcript(y_pred,input_audio):
  predict_text = ""
  pd_text = []
  temp_path=resampling_audio(input_audio)
  for idx in y_pred:
    if ASR_PRE_TRAINED_MODELS[idx]=="QuartZnet_Model":
      predict_text = quartZnetPrediction(temp_path)
      #print(predict_text)
      pd_text.append(predict_text)
      return  predict_text
    if ASR_PRE_TRAINED_MODELS[idx]=="Wav2Vec50_Model":
      predict_text = wav2VecPrediction(temp_path)
      #print(predict_text)
      pd_text.append(predict_text )
      return  predict_text
    else:
      predict_text = wav2Vec_960_Prediction(temp_path)
      return  predict_text



def meta_asr(groundTruth_path,met_model):
  wer_loss = []
  df = load_data(groundTruth_path)
  best_model = load_model(met_model)
  y_pred = predict_ASR_Model(df,best_model)
  for i in range(len(df)):
    input_audio = df['Audio'][i]
    real_text = df['GroundTruth_Text'][i]
    text  = meta_Model_transcript(y_pred,input_audio)
    wer =  evaluate_model(text,real_text)
    wer_loss.append(wer)
  return  wer_loss

def randomly_prediction(df,groundTruth_path):
  rand_wer = []
  for i in range(len(df)):
    input_audio = df['Audio'][i]
    real_text = df['GroundTruth_Text'][i]
    temp_path=resampling_audio(input_audio)
    speech2text_res = speech2textPrediction(temp_path)
    wer_speech2text = evaluate_model(speech2text_res,real_text)
    rand_wer.append(wer_speech2text)
  return rand_wer



def plot_WER(values):
  fig = go.Figure(data=go.Scatter(y=values))
  fig.update_layout(
        title='Meta WER',
        xaxis_title='Index',
        yaxis_title='Value'
    )
  fig.show()





In [53]:
# prediction with Raw Meta Model
path = "/content/gdrive/My Drive/Meta_Asr/GroundTruth.csv"
df = load_data(path)
raw_wer = meta_asr(path,RAW_META_Model)
# prediction with pre-processing Model
pre_wer = meta_asr(path,press_META_MODEL)
#prediction with randomly pre-trained mode
wer = randomly_prediction(df,path)



Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected toke

In [60]:
# Create the plot
fig = go.Figure(data=[
    go.Scatter(name='Meta ASR on raw data', y=raw_wer),
    go.Scatter(name="Meta ASR on Preprocessed data",y=pre_wer),
    go.Scatter(name='Random Model Selection', y=wer)
])

# Update the layout
fig.update_layout(
    title='Evaluation',
    xaxis_title='Audios',
    yaxis_title='WER'
)



In [ ]:
# Create DataFrame
df1 = pd.DataFrame({'Raw_wer': raw_wer, 'Pre_Processed_wer': pre_wer, 'Randomy_wer': wer})

In [ ]:
df1.to_csv('/content/gdrive/My Drive/Meta_Asr/evaluation.csv',index=0)